# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [1]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [2]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [10]:
# ваш код
from dataclasses import dataclass
from typing import Union, Callable

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", float]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value + other.value, self.d + other.d)
        return Dual(self.value + other, self.d)

    def __mul__(self, other: Union["Dual", float]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value * other.value, self.value * other.d + self.d * other.value)
        return Dual(self.value * other, self.d * other)

    __rmul__ = __mul__
    __radd__ = __add__

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

print(f_diff(2))  

def test_diff():
    f_diff = diff(f)
    assert f_diff(2) == 22.0, "Test failed for f(x) at x=2"
    print("ok")

# test
test_diff()



22.0
ok


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [31]:
# ваш код
from dataclasses import dataclass
from typing import Union, Callable
import math

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", float]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value + other.value, self.d + other.d)
        return Dual(self.value + other, self.d)

    def __mul__(self, other: Union["Dual", float]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value * other.value, self.value * other.d + self.d * other.value)
        return Dual(self.value * other, self.d * other)

    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)

    def __truediv__(self, other: Union["Dual", float]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value / other.value, (self.d * other.value - self.value * other.d) / (other.value ** 2))
        return Dual(self.value / other, self.d / other)

    def __pow__(self, exponent: Union[int, float]) -> "Dual":
        new_value = self.value ** exponent
        new_d = new_value * (exponent / self.value) * self.d if self.value != 0 else 0
        return Dual(new_value, new_d)

    def exp(self) -> "Dual":
        exp_value = math.exp(self.value)
        return Dual(exp_value, exp_value * self.d)

    def sin(self) -> "Dual":
        sin_value = math.sin(self.value)
        return Dual(sin_value, math.cos(self.value) * self.d)

    def cos(self) -> "Dual":
        cos_value = math.cos(self.value)
        return Dual(cos_value, -math.sin(self.value) * self.d)

    def log(self) -> "Dual":
        log_value = math.log(self.value)
        return Dual(log_value, self.d / self.value)

    __rmul__ = __mul__
    __radd__ = __add__

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

# test1
f_diff = diff(f)
print(f_diff(2))  #print22

def test_diff():
    f_diff = diff(f)
    assert f_diff(2) == 22.0, "Test failed for f(x) at x=2"
    print("ok！")

def test_unary_operations():
    u = Dual(2.0, 1.0)
    assert (-u).value == -2.0
    assert (-u).d == -1.0

def test_division():
    u1 = Dual(4.0, 1.0)
    u2 = Dual(2.0, 0.0)
    result = u1 / u2
    assert result.value == 2.0
    assert result.d == 0.5  # 1/2.0

def test_power():
    u = Dual(2.0, 1.0)
    result = u ** 3
    assert result.value == 8.0
    assert result.d == 12.0  # 3 * 2^2 * 1.0

def test_transcendental_functions():
    u = Dual(1.0, 1.0)  
    assert u.exp().value == math.exp(1.0)
    assert u.exp().d == math.exp(1.0)

    u = Dual(math.pi / 4, 1.0)
    assert u.sin().value == math.sin(math.pi / 4)
    assert u.sin().d == math.cos(math.pi / 4)

    assert u.cos().value == math.cos(math.pi / 4)
    assert u.cos().d == -math.sin(math.pi / 4)

    u = Dual(2.0, 1.0)
    assert u.log().value == math.log(2.0)
    assert u.log().d == 1 / 2.0

# test all
test_unary_operations()
test_division()
test_power()
test_transcendental_functions()
test_diff()


22.0
ok！


## Задание 1.3 (2 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+1 балл**)

In [46]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

ModuleNotFoundError: No module named 'scipy'

In [34]:
# ваш код
from dataclasses import dataclass
from typing import Union, Callable

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", float]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value + other.value, self.d + other.d)
        return Dual(self.value + other, self.d)

    def __mul__(self, other: Union["Dual", float]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value * other.value, self.value * other.d + self.d * other.value)
        return Dual(self.value * other, self.d * other)

    __rmul__ = __mul__
    __radd__ = __add__

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

def numerical_derivative(func: Callable[[float], float], x: float, h: float = 1e-5) -> float:
    return (func(x + h) - func(x - h)) / (2 * h)

# test
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

# count
f_diff = diff(f)

# print
print(f_diff(2))  #22

numerical_result = numerical_derivative(f, 2)
print(f"Numerical derivative at x=2: {numerical_result}")  
print(f"Analytical derivative at x=2: {f_diff(2)}") 

# test1
def test_diff():
    assert f_diff(2) == 22.0, "Test failed for f(x) at x=2"
    assert abs(numerical_derivative(f, 2) - 22.0) < 1e-5, "Numerical derivative test failed"
    print("ok！")

# test2
test_diff()


22.0
Numerical derivative at x=2: 22.000000000055305
Analytical derivative at x=2: 22.0
ok！


## Задание 1.4 (6 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [43]:
# ваш код
import ast
import random
from typing import Callable

class Dual:
    def __init__(self, value: float, d: float):
        self.value = value
        self.d = d

    def __add__(self, other):
        if isinstance(other, Dual):
            return Dual(self.value + other.value, self.d + other.d)
        return Dual(self.value + other, self.d)

    def __mul__(self, other):
        if isinstance(other, Dual):
            return Dual(self.value * other.value, self.value * other.d + self.d * other.value)
        return Dual(self.value * other, self.d * other)

    def __pow__(self, other):
        if isinstance(other, Dual):
            return Dual(self.value ** other.value, self.d * self.value ** (other.value - 1) * other.d)
        return Dual(self.value ** other, self.d * other * self.value ** (other - 1))

    __rmul__ = __mul__
    __radd__ = __add__

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    def wrapper(x):
        if isinstance(x, Dual):
            return func(x)
        return func(Dual(x, 1.0)).d
    return wrapper

def numerical_derivative(func: Callable[[float], float], x: float, h: float = 1e-5) -> float:
    return (func(x + h) - func(x - h)) / (2 * h)

def generate_random_function() -> Callable[[float], float]:
    operations = {
        '+': ast.Add,
        '-': ast.Sub,
        '*': ast.Mult,
        '/': ast.Div
    }

    def random_expr(depth=0):
        if depth > 2 or random.random() > 0.5:
            return ast.Constant(random.randint(-10, 10))
        op = random.choice(list(operations.keys()))
        left = random_expr(depth + 1)
        right = random_expr(depth + 1)
        return ast.BinOp(left=left, op=operations[op](), right=right)

    expr = ast.Expression(
        body=ast.Lambda(
            args=ast.arguments(
                args=[ast.arg(arg='x')],
                defaults=[]
            ),
            body=random_expr()
        )
    )
    ast.fix_missing_locations(expr)
    func = eval(compile(expr, filename="", mode="eval"))

    def wrapped_func(x):
        if isinstance(x, Dual):
            return func(x.value)
        return func(x)

    return wrapped_func

def test_derivatives(num_tests: int = 10):
    for _ in range(num_tests):
        func = generate_random_function()
        f_diff = diff(func)

        x = random.uniform(-10, 10)

        symbolic_result = f_diff(Dual(x, 1.0))  
        numerical_result = numerical_derivative(func, x)

        print(f"x: {x:.2f}, Symbolic: {symbolic_result:.6f}, Numerical: {numerical_result:.6f}")

        assert abs(symbolic_result - numerical_result) < 1e-5, "Test failed!"

# test
test_derivatives(20)


x: 8.39, Symbolic: -5.000000, Numerical: 0.000000


AssertionError: Test failed!

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [45]:
# ваш код
from dataclasses import dataclass
from typing import Union, Callable, List

@dataclass
class Dual:
    value: float
    derivatives: List[float]

    def __init__(self, value: float, derivatives: List[float]):
        self.value = value
        self.derivatives = derivatives

    def __add__(self, other: Union["Dual", float]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value + other.value, [d1 + d2 for d1, d2 in zip(self.derivatives, other.derivatives)])
        return Dual(self.value + other, self.derivatives)

    def __sub__(self, other: Union["Dual", float]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value - other.value, [d1 - d2 for d1, d2 in zip(self.derivatives, other.derivatives)])
        return Dual(self.value - other, self.derivatives)

    def __mul__(self, other: Union["Dual", float]) -> "Dual":
        if isinstance(other, Dual):
            new_value = self.value * other.value
            new_derivatives = [
                self.value * other.derivatives[i] + self.derivatives[i] * other.value for i in range(len(other.derivatives))
            ]
            return Dual(new_value, new_derivatives)
        return Dual(self.value * other, [d * other for d in self.derivatives])

    __rmul__ = __mul__
    __radd__ = __add__

def diff(func: Callable[..., float]) -> Callable[..., List[float]]:
    def wrapper(*args):
        n = len(args)
        dual_args = [Dual(arg, [1.0 if i == j else 0.0 for j in range(n)]) for i, arg in enumerate(args)]
        result = func(*dual_args)
        return result.derivatives
    return wrapper

def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y

f_diff = diff(f)

# test
print(f_diff(10, 10, 10)) 


[10.0, 5.0, 1.0]
